In [ ]:
from google.colab import drive
ROOT = "/content/drive"  
drive.mount(ROOT)  
%cd "/content/drive/MyDrive/PFE_PIMM/"
%ls

Mounted at /content/drive
/content/drive/MyDrive/PFE_PIMM
'Copie de labeling.ipynb'   meltpool_bitmap/     visualisation.ipynb
 ct_scan/                   registration.ipynb
 labeling.ipynb             tomo_labels/


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
from __future__ import print_function
import cv2
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from fnmatch import fnmatch
from IPython.display import clear_output

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.1

def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

def clean_vid_image(img):
    
    h, w = img.shape[:2]
    radius = h/4
    mask = create_circular_mask(h, w, radius=radius)
    masked_img = img.copy()
    masked_img[~mask] = 0
    return masked_img

def crop_vid_image(img):
    
    h, w = img.shape[:2]
    radius = h/2
    mask = create_circular_mask(h, w, radius=radius)
    masked_img = img.copy()
    cropped_img = img[np.ix_(mask.any(1), mask.any(0))]
    return cropped_img

def get_value_vid_image(img):
  i, j = np.where(img != 0)   # find indices where it is nonzero 
  value = img[i, j]           # extract nonzero values of the array
  return value

def alignImages(im1, im2,gray=True):

  # Convert images to grayscale
  if gray :
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
  else :
    im1Gray = im1
    im2Gray = im2

  im1Gray = clean_vid_image(im1Gray)

  im1Gray = cv2.medianBlur(im1Gray,3)
  im2Gray = cv2.medianBlur(im2Gray,3)

  _,im1Gray = cv2.threshold(im1Gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  _,im2Gray = cv2.threshold(im2Gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

 
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2)

  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)

  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]

  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)

  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)

  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

  
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))


  return im1Reg, h

def printProgressBar(iteration, total, prefix='', suffix='', decimals=1, length=100, fill='█', printEnd="\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}')
    # Print New Line on Complete
    if iteration == total:
        print()

In [ ]:
cap = cv2.VideoCapture('/content/drive/My Drive/PFE_PIMM/meltpool_bitmap/Piece 1.mp4')

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    clean = clean_vid_image(gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()



In [ ]:
scale_bar = cv2.imread('/content/drive/My Drive/PFE_PIMM/meltpool_bitmap/scaleBar.png')
gray = cv2.cvtColor(scale_bar, cv2.COLOR_BGR2GRAY)
max = np.amax(gray)
min = np.amin(gray)

def map_scale(x, in_min=189, in_max=58, out_min=-0.15, out_max=1.33):
  return ((out_max - out_min)/(float(in_max) - float(in_min)))*(float(x) - float(in_max)) + out_max



In [ ]:
frame = cv2.imread('/content/drive/My Drive/PFE_PIMM/meltpool_bitmap/slice_part3.png')

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

cropped = crop_vid_image(gray)

value = get_value_vid_image(cropped)

print(value.shape)

(435016,)


In [ ]:
image = cv2.imread('/content/drive/My Drive/PFE_PIMM/meltpool_bitmap/Piece 1.mp4')

height, width, channels = image.shape
img_binary = np.zeros((height, width, 1))
img_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

ret, thresh = cv2.threshold(img_grayscale, 127, 255, 0)
contours, hierarchy = cv2.findContours(thresh, 1, 2)

largest_contour_area = 0
largest_contour_area_idx = 0

for i in range(len(contours)):
    if (cv2.contourArea(contours[i]) > largest_contour_area):
        largest_contour_area = cv2.contourArea(contours[i])
        largest_contour_area_idx = i

hull = cv2.convexHull(contours[largest_contour_area_idx])

# display results
cv2.drawContours(image, [hull], -1, (0, 255, 0))
cv2_imshow(image)


mask = np.zeros(img.shape, np.uint8)
cv2.drawContours(mask, hull, -1, (255),1)
locs = np.where(mask == 255)
pixels = image[locs]

pixels

In [ ]:
# video to png frames
cap= cv2.VideoCapture('/content/drive/My Drive/PFE_PIMM/meltpool_bitmap/Piece 1.mp4')
i = 0
fps, frames = cap.get(cv2.CAP_PROP_FPS), cap.get(cv2.CAP_PROP_FRAME_COUNT)
printProgressBar(0, int(frames), prefix='Extracting frames : ', suffix='Complete', length=50)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('/content/drive/My Drive/PFE_PIMM/meltpool_bitmap/Frame/frame_'+str(i)+'.png',frame)
    clear_output(wait = True)
    printProgressBar(i + 1, int(frames), prefix='Extracting frames : ', suffix='Complete', length=50)
    i+=1

Creating video :  |██████████████████████████████████████████████████| 100.0% Complete



In [ ]:
# Read reference image
refFilename = "data/feet.png"
print("Reading reference image : ", refFilename)
imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

Reading reference image :  data/feet.png


In [ ]:
imFilename = "data/frame_7.png"
print("Reading image to align : ", imFilename);
im = cv2.imread(imFilename, cv2.IMREAD_COLOR)


Reading image to align :  data/frame_7.png


In [ ]:
print("Aligning images ...")
# Registered image will be resotred in imReg.
# The estimated homography will be stored in h.
imReg, h = alignImages(im, imReference,gray=True)
# Write aligned image to disk.
outFilename = "aligned.jpg"
print("Saving aligned image : ", outFilename);
cv2.imwrite(outFilename, imReg)

# Print estimated homography
print("Estimated homography : \n",  h)

Aligning images ...
Saving aligned image :  aligned.jpg
Estimated homography : 
 [[ 3.56328527e+00  1.17053896e-02 -2.63622206e+03]
 [ 7.89312192e-02  3.62951700e+00 -1.47840095e+03]
 [ 1.08667357e-04  5.47643424e-05  1.00000000e+00]]


In [ ]:
# APPLYING HOMOGRAPHY TO REST OF FRAMES 

fourcc = cv2.VideoWriter_fourcc(*'XVID')
cap= cv2.VideoCapture('data/Piece 1.mp4')
height, width, channels = imReference.shape
fps, frames = cap.get(cv2.CAP_PROP_FPS), cap.get(cv2.CAP_PROP_FRAME_COUNT)
out = cv2.VideoWriter('result.mp4', fourcc, fps, (width, height))
listOfFrames = ['./data/'+x for x in listdir('./data/') if fnmatch(x, "frame_*")]
printProgressBar(0, len(listOfFrames), prefix='Creating video : ', suffix='Complete', length=50)
for index,frame in enumerate(listOfFrames) :
    frame = cv2.imread(frame)
    im_buffer = cv2.warpPerspective(frame, h, (width, height))
    out.write(im_buffer)
    clear_output(wait = True)
    printProgressBar(index + 1, len(listOfFrames), prefix='Creating video : ', suffix='Complete', length=50)
cap.release()
out.release()



Creating video :  |██████████████████████████████████████████████████| 100.0% Complete

